In [1]:
from src.data.dataset import SoundDS
sd= SoundDS('')
# sd.load('/home/patrick/Workspace/Projects/SHT7/AutoPitching/data/evil/Krung Thon Buri Frontage Rd. 2.m4a', 1)

In [2]:
sd.add('/home/patrick/Workspace/Projects/SHT7/AutoPitching/data/evil', 1)

In [3]:
sd.add('/home/patrick/Workspace/Projects/SHT7/AutoPitching/data/heaven', 0)

In [4]:
len(sd)

34201600

In [5]:
from src.helper.train import train
from src.helper.inference import validate
import torch
train_set, valid_set = torch.utils.data.random_split(sd, [int(len(sd)*0.8), len(sd)-int(len(sd)*0.8)])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=8, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=8, shuffle=False)

In [6]:

from src.model.head import RawHead
model =  RawHead(80,8,1)
optim = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = 'cpu'

In [7]:
model.train()
model.to(device)
total_loss = 0
accum_size = 1
start_epoch = 1
num_epoch = 3

for epoch in range(start_epoch, num_epoch + 1):
  print(f'Epoch {epoch}')
  for data, target in train_loader:
      print(data.shape)
      data, target = data.to(device), target.to(device)
      print(data.shape)
      optim.zero_grad()
      output = model(data)
      print(output)
      loss = torch.nn.functional.cross_entropy(output,target) / accum_size
      total_loss += loss.item() * accum_size
      loss.backward(retain_graph=True)
      optim.step()
      print(f'Loss {loss.item()}',end="\r")
  print('Total Loss', total_loss)
  if scheduler: scheduler.step()

Epoch 1


: 

: 